### Парсинг на Python:

Для начала я хочу написать, что я всего лишь 2 раза занимался такой штукой и сейчас решил изучить ее поподробнее. Для начала научусь работать с библиотекой BeatifulSoup и сделаю небольшую систему оповещений и она будет состоять из следующих тасков, которые будут обернуты в dag:
1. Паршу вакансии(практически закончил разобраться почему порой он берет значения не от туда, а также стоило бы добавить описание, чтобы сообщения отправлялись с пользой);
2. Загружаю значения в бд;
3. Беру только новые значения, а то есть left-only join;
4. Отправляю все новые значения в телеграм канал каждый день, при этом также можно будет подумать в каком формате отсылать сообщения и в какие графики.

In [543]:
#Для датафрейма:
import pandas as pd
import numpy as np

#Для визуализации:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#Для парсинга:
import requests
from hyper.contrib import HTTP20Adapter
from bs4 import BeautifulSoup as bs
from time import sleep

#Для airflow и clickhouse:
from airflow.decorators import dag, task
from datetime import datetime, timedelta
from clickhouse_driver import Client

#Для бота:
import telegram
import io

Парсить я буду квартиры в Москве с сайта авито, далее я хочу поисследовать данные и сделать какие-то выводы.

In [331]:
url = "https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p=1"

In [332]:
s = requests.Session()
s.mount('https://', HTTP20Adapter())
r = s.get(url)
r.encoding = 'utf-8'
print(r.status_code)

200


In [333]:
data = bs(r.text, 'lxml')

In [334]:
flats = data.findAll("div", class_="iva-item-content-rejJg")
len(flats)

53

1. Код 200 означает что все прошло успешно.
2. Парсить буду следующее:
    - Метро; +
    - Цену на квартиру;
    - Цену за квадратный метро;
    - Ссылку на квартиру. +

In [335]:
subway = data.find('div', class_="geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL").text
subway

'Коммунарка21–30\xa0мин.'

In [336]:
needed_part_subway = ""
for i in subway:
    if i.isalpha() == True:
        needed_part_subway += i
    else:
        break

In [337]:
needed_part_subway

'Коммунарка'

Метро будем искать так, конечно - это будет немного долго, так как каждый раз будет отрабатывать такой цикл, но после я перепишу выражение используя re.

In [338]:
link = data.find("a",
                 "link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH").get(
    "href")

In [339]:
full_link = "https://www.avito.ru" + link
full_link

'https://www.avito.ru/moskva/kvartiry/2-k._kvartira_496m_517et._2338890812'

А теперь разберемся с ценой на квартиру:

In [340]:
full_price = data.find("span", class_="price-text-_YGDY text-text-LurtD text-size-s-BxGpL").text

In [341]:
correct_number_full_price = ""
for i in full_price:
    if i.isdigit() == True:
        correct_number_full_price += i

In [342]:
correct_number_full_price

'10177920'

Осталось разобраться с ценой на квадратный метр:

In [343]:
price_for_square_metre = data.find("span",
                                   class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL").text

In [344]:
correct_number_square_price = ""

In [345]:
for i in price_for_square_metre:
    if i.isdigit() == True:
        correct_number_square_price += i

In [346]:
correct_number_square_price = correct_number_square_price.replace(correct_number_square_price[-1], "")
correct_number_square_price

'205200'

Теперь все это стоит поместить в цикл, но пока просто протестируем:

In [544]:
lst_links = []
lst_square = []
lst_price = []
lst_subway = []

In [545]:
def parce_vacancies():

    #Сначала создаем цикл, где будем идти по страницам:
    for i in range(1, 99):
        print(i)#чтобы отслеживать на какой страницы мы
        url = f"https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p={i}"
        s = requests.Session()
        s.mount('https://', HTTP20Adapter())
        r = s.get(url)
        sleep(7) #чтобы не перегружался
        #инкодим значения иначе будет эльфийский
        r.encoding = 'utf-8'
        data = bs(r.text, 'lxml')

        flats = data.findAll("div", class_="iva-item-content-rejJg")


        #А здесь уже большой цикл обработки, где будем идти и собирать данные с каждой квартиры:
        for flat in flats:

            #Ссылка на квартиру:
            var_link = flat.find("a",
                                 "link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH")
            if type(var_link) != type(None):
                local_var_link = "https://www.avito.ru" + var_link.get("href")
                lst_links.append([local_var_link])

            #Отбираем цену за квадрат:
            if type(flat.find("span",
                              class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL")) != type(
                None):
                var = flat.find("span",
                                class_="price-noaccent-X6dOy price-normalizedPrice-PplY9 text-text-LurtD text-size-s-BxGpL").text
                correct_number_square_price = ""
                #Очищаем строку от бесполезного мусора:
                for i in var:
                    if i.isdigit() == True:
                        correct_number_square_price += i
                #Убираем последнею цифру, там значение в квадрате:
                correct_number_square_price = correct_number_square_price.replace(correct_number_square_price[-1], "")
                lst_square.append(correct_number_square_price)
            else:
                continue

            #Отбираем полную цену:
            var_full_price = flat.find("span", class_="price-text-_YGDY text-text-LurtD text-size-s-BxGpL")
            if type(var_full_price) != type(None):
                full_price = var_full_price.text
                correct_number_full_price = ""
                #Оставляем лишь число:
                for i in full_price:
                    if i.isdigit() == True:
                        correct_number_full_price += i

                lst_price.append(correct_number_full_price)
            else:
                lst_price.append(None)

            #Отбираем станцию метро:
            var_subway = flat.find('div', class_="geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL")
            if type(var_subway) != type(None):
                subway_name = var_subway.text
                subway_full = ""
                for i in subway_name:
                    if i.isalpha() == True:
                        subway_full += i
                    else:  #после станции идет время сколько от метро и это значение число, тем самым убираем его
                        break
                lst_subway.append(subway_full)

            else:
                lst_subway.append(None)

    return lst_links, lst_square, lst_price, lst_subway


In [546]:
parce_vacancies()

1
[2022-08-31 23:34:33,980] {connection.py:724} INFO - Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=SettingCodes.MAX_CONCURRENT_STREAMS, original_value=None, new_value=16), ChangedSetting(setting=SettingCodes.INITIAL_WINDOW_SIZE, original_value=65535, new_value=65536), ChangedSetting(setting=SettingCodes._max_frame_size, original_value=16384, new_value=16777215)}>
[2022-08-31 23:34:33,982] {connection.py:724} INFO - Received unhandled event <WindowUpdated stream_id:0, delta:2147418112>
[2022-08-31 23:34:33,996] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{ChangedSetting(setting=SettingCodes.ENABLE_PUSH, original_value=1, new_value=0)}>
[2022-08-31 23:34:33,998] {connection.py:724} INFO - Received unhandled event <SettingsAcknowledged changed_settings:{}>
2
[2022-08-31 23:34:43,882] {connection.py:724} INFO - Received unhandled event <RemoteSettingsChanged changed_settings:{ChangedSetting(setting=

([['https://www.avito.ru/moskva/kvartiry/3-k._kvartira_699m_1524et._2532072536'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_486m_216et._2540765073'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_442m_330et._2454865841'],
  ['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_323m_19et._2534134115'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_624m_925et._2498940272'],
  ['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_341m_2225et._2498863842'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_422m_69et._2534386569'],
  ['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_423m_633et._2498998824'],
  ['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_348m_912et._2550731565'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_413m_1033et._2507907046'],
  ['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_365m_616et._2552723280'],
  ['https://www.avito.ru/moskva/kvartiry/2-k._kvartira_526m_2633et._2498923244'],
  ['https://www.avito.ru/m

In [547]:
display(len(lst_links), len(lst_square), len(lst_price), len(lst_subway))

4900

4891

4891

4891

In [551]:
headers = ["square_price","full_price","subway"]

In [569]:
df_rest = pd.DataFrame(data={"square_price":lst_square,"full_price":lst_price,"subway":lst_subway})
df_rest.head()

,square_price,full_price,subway
0,434192,30350000,Кунцевская
1,425930,20700180,ЗИЛ
2,230597,10192402,Бунинская
3,92879,3000000,Строгинодо
4,188100,11737440,Братиславская


In [567]:
df_links = pd.DataFrame(data={"links":lst_links})

In [571]:
df_all = pd.concat([df_links,df_rest],axis=1)